In [8]:
#!pip install numpy
#!pip install pandas
#!pip install seaborn
#!pip install matplotlib
#!pip install scikit-learn

### Загружаем необходимые библиотеки

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Библиотека для разбиения датасета на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
#Библиотека для подбора лучших параметров модели
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge

#Библиотека для обучния модели с несколькими выходами
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score


### Загружаем датасет и исследуем его

In [14]:
# Загружаю датасет, записываю его в переменные X и y. 
dataset = pd.read_csv('ebw_data.csv')
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ebw_data.csv'

In [15]:
#Размер датасета
dataset.shape

NameError: name 'dataset' is not defined

In [ ]:
#Проверяю, есть ли пустые значения
dataset.isna().sum()

In [ ]:
#Строю графики для оценки данных датасета
sns.pairplot(dataset) 

In [ ]:
#По графикам видно, что столбцы IF и IW, IF и FP коррелируют между собой. Матрица корреляции это подтверждает
dataset.corr()

In [ ]:
#Более наглядно на тепловой карте корреляции датасета
sns.heatmap(dataset.corr())

### Подготовка данных для обучения

In [ ]:
#Разделяю датасет на параметры и предсказываемые значения (таргеты)
X = dataset.drop(['Depth','Width'], axis=1).values
y = dataset[['Depth','Width']].values
dataset_names = ['IW',	'IF',	'VW',	'FP']

In [ ]:
#Разбиваю датасет на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

### Выбор наилучшей модели для обучения

In [ ]:
#создаю список для перебора вариантов моделей
regressors = [
    GradientBoostingRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
    Ridge()
]

In [ ]:
#Перебираем в цикле модели и выбираем лучшую модель
head = 5 
for model in regressors[:head]:
    multiout = MultiOutputRegressor(model).fit(X_train, y_train)
    y_pred = multiout.predict(X_test)
    print(model)
    print("\tMean squared error:", MultiOutputRegressor(mean_squared_error(y_test, y_pred)))
    print("\tR2 score:", MultiOutputRegressor(r2_score(y_test, y_pred)))
    print()

### Выбираю для дальнейшего обучения GradientBoostingRegressor

In [ ]:
#Выбираю для дальнейшего исследования GradientBoostingRegressor
#Так как показатели этой модели наилучшие при параметрах по-умолчанию
#Пробую подобрать параметры, при которых модель будет работать еще лучше
#Создаю словарь с параметрами для поиска лучшего решения
parameters = { 'n_estimators' :[70, 100, 120],
              'max_depth':[2,3,5], # установка глубины дерева 
              'max_features':[2,3,4], # количество признаков для построение разветвлений
              'min_samples_leaf': [1,2,3], #минимальное число объектов в листе
              'min_samples_split':[2,3,5,7,10] #минимальное число объектов, при котором происходит разветвление
             }

In [ ]:
#Обучаем модель отдельно для Depth и Width. 
grid = GradientBoostingRegressor(criterion = "squared_error")
gridCV = GridSearchCV(grid,
                      parameters,
                     )
model_Depth = gridCV.fit(X, y[:,0]) #Отдельно обучение для Depth
params_Depth = model_Depth.best_params_ #Лучшие параметры для Depth
print('Лучшие параметры для Depth', params_Depth,'\n')
model_Width = gridCV.fit(X, y[:,1])  #Отдельно обучение для Width
params_Width = model_Width.best_params_ #Лучшие параметры для Width
print('Лучшие параметры для Width', params_Width,'\n')

In [ ]:
#Подставляю подобранные параметры в модель для Depth  
GBR_Depth  = GradientBoostingRegressor(**params_Depth).fit(X_train, y_train[:,0])
y_pred_best_Depth = GBR_Depth.predict(X_test)

In [ ]:
#Подставляю подобранные параметры в модель для Width
GBR_Width  = GradientBoostingRegressor(**params_Width).fit(X_train, y_train[:,1])
y_pred_best_Width = GBR_Width.predict(X_test)

### Оценка качества модели

In [ ]:
#Среднеквадратичная ошибка и коэффициент детерминации для Depth
mse_Depth = mean_squared_error(y_test[:,0], y_pred_best_Depth)
print("MSE_Depth on test set: {:.4f}".format(mse_Depth))
R2_Depth = r2_score(y_test[:,0], y_pred_best_Depth)
print("R2_Depth on test set: {:.4f}".format(R2_Depth))

#Среднеквадратичная ошибка и коэффициент детерминации для Width
mse_Width = mean_squared_error(y_test[:,1], y_pred_best_Width)
print("MSE_Width on test set: {:.4f}".format(mse_Width))
R2_Width = r2_score(y_test[:,1], y_pred_best_Width)
print("R2_Width on test set: {:.4f}".format(R2_Width))

In [ ]:
#Построю графики обучения на параметрах для Depth
test_score = np.zeros((params_Depth["n_estimators"],), dtype=np.float64)
for i, y_pred_best_Depth in enumerate(GBR_Depth.staged_predict(X_test)):
    test_score[i] = GBR_Depth.loss_(y_test[:,0], y_pred_best_Depth)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params_Depth["n_estimators"]) + 1,
    GBR_Depth.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params_Depth["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance for Depth")
fig.tight_layout()
plt.show()

In [ ]:
#Построю графики обучения на параметрах для Width
test_score = np.zeros((params_Width["n_estimators"],), dtype=np.float64)
for i, y_pred_best_Width in enumerate(GBR_Width.staged_predict(X_test)):
    test_score[i] = GBR_Width.loss_(y_test[:,1], y_pred_best_Width)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params_Width["n_estimators"]) + 1,
    GBR_Width.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params_Width["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance for Width")
fig.tight_layout()
plt.show()